### 2025-10-24 Live Sessione 1, Part 1

### Exercise 2021.1 – fit a model to unbinned dataset

Create a Gaussian p.d.f. with mean = 0, and sigma = 1. 
Visualize the p.d.f. Change the sigma to 3.  
Generate an **unbinned** dataset of 10000 events.
Make a Fit with Maximum Likelihood. Visualize the results.

*Tips:*
 - You can follow the comments inside the macro **roofit\_empty.cpp**
   or **roofit_empty.ipynb**
 - *Use information from the slides shown during the lecture or from
    RooFit Manual at par 2*
 - *Refer to the tutorial rf101\_basics.cxx*

In [21]:
import ROOT

In [22]:
# S e t u p   m o d e l
# ---------------------

# Declare variables x,mean,sigma with associated name, title, initial value and allowed range

x = ROOT.RooRealVar("x", "x", -10, 10)
mean = ROOT.RooRealVar("mean", "mean of gaussian", 0, -10, 10)
sigma = ROOT.RooRealVar("sigma", "width of gaussian", 1, 0.00001, 10)
# WARNING: don't set the lower limit of sigma exactly equal to 0

# Build gaussian pdf in terms of x,mean and sigma
gauss = ROOT.RooGaussian("gauss", "gaussian PDF", x, mean, sigma); 
# NOTE: no need a third parameter for the height. RooFit automatically takes care about normalization.


In [23]:
# P l o t   m o d e l   a n d   c h a n g e   p a r a m e t e r   v a l u e s
# ---------------------------------------------------------------------------

# Construct plot frame in 'x'
xframe = x.frame()   # return a RooPlot

# Plot gauss in frame (i.e. in x)
gauss.plotOn(xframe)  # Note: the model is normalized to 1

# Change the value of sigma to 3 
sigma.setVal(3)

# Plot gauss in frame (i.e. in x) and draw frame on canvas
gauss.plotOn(xframe, ROOT.RooFit.LineColor(ROOT.kRed))

c = ROOT.TCanvas("c", "c", 800, 600)
xframe.Draw()
c.Draw()

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c


In [24]:
# G e n e r a t e   e v e n t s
# -----------------------------

# Generate a dataset of 1000 events in x from gauss
data = gauss.generate(x, 1000)

# Make a second plot frame in x and draw both the
# data and the pdf in the frame
xframe2 = x.frame()    # return a RooPlot* (pointer)
data.plotOn(xframe2)   # Warning: plot the the data before the model!  
gauss.plotOn(xframe2)  # Note: the model is normalized to the n. of events of the dataset


In [25]:
# F i t   m o d e l   t o   d a t a
# -----------------------------

# Fit pdf to data
gauss.fitTo(data)

# Print values of mean and sigma (that now reflect fitted values and errors)
mean.Print()
sigma.Print()

# Draw data and model in a new frame
c = ROOT.TCanvas("c", "c", 800, 600)
xframe2.Draw()
c.Draw()


[#1] INFO:Fitting -- RooAbsPdf::fitTo(gauss_over_gauss_Int[x]) fixing normalization set for coefficient determination to observables in data
[#1] INFO:Fitting -- Creation of NLL object took 428.9 μs
[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_gauss_over_gauss_Int[x]_gaussData) Summation contains a RooNLLVar, using its error level
[#1] INFO:Minimization -- RooAbsMinimizerFcn::setOptimizeConst: activating const optimization
Minuit2Minimizer: Minimize with max-calls 1000 convergence for edm < 1 strategy 1
Minuit2Minimizer : Valid minimum - status = 0
FVAL  = 2490.41048070331453
Edm   = 0.000213663529107514669
Nfcn  = 24
mean	  = 0.108706	 +/-  0.0931053	(limited)
sigma	  = 2.93223	 +/-  0.0672778	(limited)
[#1] INFO:Minimization -- RooAbsMinimizerFcn::setOptimizeConst: deactivating const optimization
RooRealVar::mean = 0.108706 +/- 0.0931051  L(-10 - 10) 
RooRealVar::sigma = 2.93223 +/- 0.0672777  L(1e-05 - 10) 


Info in <Minuit2>: MnSeedGenerator Computing seed using NumericalGradient calculator
Info in <Minuit2>: MnSeedGenerator Evaluated function and gradient in 757.8 μs
Info in <Minuit2>: MnSeedGenerator Initial state: FCN =       2491.526206 Edm =       1.103517502 NCalls =      9
Info in <Minuit2>: MnSeedGenerator Initial state  
  Minimum value : 2491.526206
  Edm           : 1.103517502
  Internal parameters:	[                0    -0.4115183736]	
  Internal gradient  :	[     -119.5436213      61.16148407]	
  Internal covariance matrix:
[[  0.00018187162              0]
 [              0  0.00048520015]]]
Info in <Minuit2>: VariableMetricBuilder Start iterating until Edm is < 0.001 with call limit = 1000
Info in <Minuit2>: VariableMetricBuilder    0 - FCN =       2491.526206 Edm =       1.103517502 NCalls =      9
Info in <Minuit2>: VariableMetricBuilder    1 - FCN =       2490.410481 Edm =   0.0002378161241 NCalls =     14
Info in <Minuit2>: MnHesse Done after 808.8 μs
Info in <Minuit2>

### 2025-10-24 Live Sessione 1, Part 2

### Exercise 2021.2 – fit a model to binned dataset

Create an Exponential p.d.f. to with rate = -1/tau where tau = 4.5  is the mean life. Visualize the p.d.f. . Generate a **binned** dataset of 1000 events **(bin width = 0.5)**.  Make a Fit with Maximum Likelihood. Visualize the results.

_Tips:_
<br>\- You can follow the comments inside the macro **roofit\_empty.cpp** or **roofit_empty.ipynb** 
<br>\- _Define the mean life as a RooRealVar and express the exponential rate using RooFormulaVar_
<br>\- _The binning of the returned RooDataHist is controlled by the default binning associated with the observables generated. To set the number of bins in x to 200, do e.g. x.setBins(200) prior to the call to generateBinned()_


In [26]:
# just do minimal changes to the previous exercise

# - define the tau parameter with starting value 4.5
tau = ROOT.RooRealVar("tau", "mean life", 4.5, 0.5, 10)

# - exponential rate is a derived quantities, which we express as a equation
#   the second argument contains the formula formatted as done on C++
alpha = ROOT.RooFormulaVar("alpha", "-1./@0", ROOT.RooArgList(tau))

# - adjust the observable range, since the pdf make no sense for negative values
x.setRange(0, 10)

# - define the exponential pdf 
model2 = ROOT.RooExponential("model2", "model2", x, alpha)

# - set the bin number as required
x.setBins(20)


In [27]:
# draw (as done before...)

xframe3 = x.frame()
model2.plotOn(xframe3)  

c = ROOT.TCanvas()
xframe3.Draw()
c.Draw()

In [34]:
# - generate a BINNED dataset and fit

data2 = model2.generateBinned(x, 1000)   # <-- replace genrate(..) with generateBinned(..)

model2.fitTo(data2)                      # the command is unchanged, but the dataset is binned now

<cppyy.gbl.RooFitResult object at 0x(nil)>

[#1] INFO:Fitting -- RooAbsPdf::fitTo(model2_over_model2_Int[x]) fixing normalization set for coefficient determination to observables in data
[#1] INFO:Fitting -- Creation of NLL object took 890.8 μs
[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_model2_over_model2_Int[x]_genData) Summation contains a RooNLLVar, using its error level
[#1] INFO:Minimization -- RooAbsMinimizerFcn::setOptimizeConst: activating const optimization
Minuit2Minimizer: Minimize with max-calls 500 convergence for edm < 1 strategy 1
Minuit2Minimizer : Valid minimum - status = 0
FVAL  = 2108.9955055347441
Edm   = 1.23769437169738794e-08
Nfcn  = 13
tau	  = 4.36373	 +/-  0.235549	(limited)
[#1] INFO:Minimization -- RooAbsMinimizerFcn::setOptimizeConst: deactivating const optimization


Info in <Minuit2>: MnSeedGenerator Computing seed using NumericalGradient calculator
Info in <Minuit2>: MnSeedGenerator Evaluated function and gradient in 185.1 μs
Info in <Minuit2>: MnSeedGenerator Initial state: FCN =       2108.996258 Edm =   0.0007513221874 NCalls =      3
Info in <Minuit2>: MnSeedGenerator Initial state  
  Minimum value : 2108.996258
  Edm           : 0.0007513221874
  Internal parameters:	[    -0.1896304325]	
  Internal gradient  :	[    -0.7716618752]	
  Internal covariance matrix:
[[   0.0050469862]]]
Info in <Minuit2>: VariableMetricBuilder Start iterating until Edm is < 0.001 with call limit = 500
Info in <Minuit2>: VariableMetricBuilder    0 - FCN =       2108.996258 Edm =   0.0007513221874 NCalls =      3
Info in <Minuit2>: VariableMetricBuilder    1 - FCN =       2108.995506 Edm =    2.02266497e-08 NCalls =      6
Info in <Minuit2>: MnHesse Done after 46 μs
Info in <Minuit2>: VariableMetricBuilder After Hessian
Info in <Minuit2>: VariableMetricBuilder    2

In [35]:
# draw (as done before... nothing changes even if the dataset is binned now) 

# Don't forget: draw first the data and then the model

xframe4 = x.frame()
data2.plotOn(xframe4)
model2.plotOn(xframe4)  

c = ROOT.TCanvas()
xframe4.Draw()
c.Draw()